In [1]:
import requests
import pandas as pd
from itertools import combinations

import time

In [ ]:
import time
import requests
import pandas as pd
from itertools import combinations

# Load the POI data
df = pd.read_csv('FEB_15_POI_DATA_CENTERED_2KM_DUMO.csv')
#df = df.head(10)

# Define the OSRM server URL
osrm_base_url = "http://127.0.0.1:5000"

# Initialize a list to store distances
distances = []


#http://localhost:5000/route/v1/foot/{lon1},{lat1};{lon2},{lat2}?overview=false

# Iterate through all unique pairs of POIs
for (i, row1), (j, row2) in combinations(df.iterrows(), 2):
    time.sleep(0.1)  # Add a small delay to avoid overwhelming the server
    
    # Extract coordinates for the two POIs
    coords1 = f"{row1['LONGITUDE']},{row1['LATITUDE']}"
    coords2 = f"{row2['LONGITUDE']},{row2['LATITUDE']}"

    # Build the OSRM query URL
    osrm_url = f"{osrm_base_url}/route/v1/foot/{coords1};{coords2}?overview=false"

    # Query OSRM
    response = requests.get(osrm_url)

    # Parse the response
    if response.status_code == 200:
        data = response.json()
        # Extract distance (in meters)
        distance = data['routes'][0]['distance']
        # Extract duration (in seconds)
        duration = data['routes'][0]['duration']
        # Append to the distances list
        distances.append({
            'From_POI': row1['POI'],
            'From_Type': row1['TYPE'],
            'From_wiki': row1['WIKIDATA'],
            'To_POI': row2['POI'],
            'To_Type': row2['TYPE'],
            'To_wiki': row2['WIKIDATA'],
            'Distance (m)': distance,
            'Duration (s)': duration
        })
    else:
        print(f"Error: Could not calculate route from {row1['POI']} to {row2['POI']}")

# Convert the distances list to a DataFrame
distance_matrix_df = pd.DataFrame(distances)

# Save the results to a CSV file
distance_matrix_df.to_csv("poi_distances_with_types_and_wikidata_POI_final.csv", index=False)

# Optionally print a sample of the DataFrame
distance_matrix_df